In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import learn2learn as l2l
from model import *
import matplotlib.pyplot as plt 

# Old and new classes

In [17]:
labels11 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-11LABEL.csv"
labels4 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-4LABEL.csv"

labels10 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-10LABEL.csv"
labels5 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-5LABEL.csv"

labels9 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-9LABEL.csv"
labels6 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-6LABEL.csv"

labels4_7 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-Combined_4_7.csv"
labels4_8 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-Combined_4_8.csv"
labels5_6 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-Combined_5_6.csv"
labels5_7 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-Combined_5_7.csv"
labels6_5 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-Combined_6_5.csv"
labels6_6 = "/home/ids/NT505.O21/Datasets/Changed_dataset/CICIDS2017-Combined_6_6.csv"

In [18]:
data_old = pd.read_csv(labels10)
print(data_old["Label"].unique())

[ 0  1  2  3  4  6  9  5 13 14]


In [19]:
# X_train, X_test = train_test_split(data_old, test_size=0.5)

# Initial learning

In [20]:
ways = 5
shots = 5
meta_lr = 0.01
fast_lr = 0.1
adaptation_steps = 1
meta_batch_size = 8
# device = torch.device('cpu')
# cuda = False
# if cuda:
#     torch.cuda.manual_seed(42)
#     device = torch.device('cuda')

In [21]:
My_train_old = MyDataset(name="Full_data10", dataframe=data_old, label_column="Label")
print(My_train_old.__getitem__(2))

(tensor([ 2.3236e+06,  5.3000e+01,  5.9674e+04,  4.0000e+00,  2.0000e+00,
         1.9600e+02,  2.2000e+02,  4.9000e+01,  4.9000e+01,  4.9000e+01,
         0.0000e+00,  1.1000e+02,  1.1000e+02,  1.1000e+02,  0.0000e+00,
         6.9712e+03,  1.0055e+02,  1.1935e+04,  1.6367e+04,  3.1223e+04,
         4.0000e+00,  3.1231e+04,  1.0410e+04,  1.8024e+04,  3.1223e+04,
         4.0000e+00,  4.0000e+00,  4.0000e+00,  0.0000e+00,  4.0000e+00,
         4.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.2800e+02,  4.0000e+01,  6.7031e+01,  3.3515e+01,  4.9000e+01,
         1.1000e+02,  6.6429e+01,  2.9765e+01,  8.8595e+02,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  7.7500e+01,  4.9000e+01,
         1.1000e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  4.0000e+00,  1.9600e+02,  2.0000e+00,
         2.2000e+02, -1.0000e+00, -1.0000e+00,  3.

In [22]:
meta_data_old =l2l.data.MetaDataset(My_train_old)
print(len(meta_data_old))
print(type(meta_data_old))

2817253
<class 'learn2learn.data.meta_dataset.MetaDataset'>


## Split tasks

In [23]:
task_trans = [
    l2l.data.transforms.NWays(dataset=meta_data_old, n=ways),          # N-way, K-shot
    l2l.data.transforms.KShots(dataset=meta_data_old, k=shots * 2),
    l2l.data.transforms.LoadData(dataset=meta_data_old),
]
task_train_old = l2l.data.TaskDataset(dataset = meta_data_old,
                                      task_transforms = task_trans, num_tasks=50000)

In [24]:
count = 0
for task in task_train_old:
    if count < 1:
        X, y = task
        print(X.shape)
        print(y.shape)
        count += 1
    else:
        break
# X, y = task_train_old.sample()

torch.Size([50, 78])
torch.Size([50])


In [25]:
def accuracy(predictions, targets):
    # print(targets.size(0))
    # print(targets.shape)
    predictions = predictions.argmax(dim = 1).view(targets.shape)
    return (predictions.round() == targets).sum().float() / targets.size(0)

In [26]:
def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways):
    data, labels = batch
    # data, labels = data.to(device), labels.to(device)

    # Seperate data into adaptation and evaluation sets
    adaptation_indices = np.zeros(data.size(0), dtype = bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)

    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        # print(learner(adaptation_data).shape)
        # print(adaptation_labels.shape)
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)
    
    # Evaluatie the adapted model
    predictions = learner(evaluation_data)
    # print("Logit value: {}".format(predictions))
    evaluation_error = loss(predictions, evaluation_labels)
    evaluation_accuracy = accuracy(predictions, evaluation_labels)
    return evaluation_error, evaluation_accuracy

In [27]:
def train_model(maml_dnn, meta_batch_size, task_num, epochs, opt, loss, adaptation_steps, shots, ways):
    for iteration in range(epochs):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_acc = 0.0

        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml_dnn.clone()
            # print(len(task11))
            # print(len(task11.sample()))
            batch = task_num.sample()
            evaluation_error, evaluation_acc = fast_adapt(batch, learner, loss, adaptation_steps, shots, ways)
            evaluation_error.backward()
            
            meta_train_error += evaluation_error.item()
            meta_train_acc += evaluation_acc.item()

        meta_train_acc_sum = meta_train_acc / meta_batch_size
        meta_train_error_sum = meta_train_error / meta_batch_size

        print(f"Iteration {iteration}")
        print('Meta train Accuracy: ', end="")
        print(f'{meta_train_acc_sum}')
        print('\n')

        # Average the accumulated gradients and optimize
        for para in maml_dnn.parameters():
            para.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

In [28]:
def test_model(maml_dnn, meta_batch_size, task_num, loss, adaptation_steps, shots, ways):
    meta_test_acc = 0.0
    meta_test_error = 0.0
    for task in range(meta_batch_size):
        # Compute meta-testing loss
        learner = maml_dnn.clone()
        batch = task_num.sample()
        evaluation_error, evaluation_acc = fast_adapt(batch, learner, loss, adaptation_steps, shots, ways)  
        
        meta_test_acc += evaluation_acc.item()
        meta_test_error += evaluation_error.item()
        
    meta_test_acc_sum = meta_test_acc / meta_batch_size
    meta_test_error_sum = meta_test_error / meta_batch_size

    print(f"Meta task Accuracy: ", end='')
    print(f"{meta_test_acc_sum}")

In [29]:
dnn_model1 = DFNN(input_shape=78, num_classes=15)
maml_dnn1 = l2l.algorithms.MAML(dnn_model1, lr = fast_lr, first_order=False)

opt = optim.Adam(maml_dnn1.parameters(), meta_lr)
loss = nn.CrossEntropyLoss()

## Meta-training

In [30]:
train_model(maml_dnn1, meta_batch_size, task_train_old, 10000, opt, loss, adaptation_steps, shots, ways)

Iteration 0
Meta train Accuracy: 0.1900000013411045


Iteration 1
Meta train Accuracy: 0.1849999986588955


Iteration 2
Meta train Accuracy: 0.18999999947845936


Iteration 3
Meta train Accuracy: 0.20000000018626451


Iteration 4
Meta train Accuracy: 0.21499999798834324


Iteration 5
Meta train Accuracy: 0.17999999970197678


Iteration 6
Meta train Accuracy: 0.2000000011175871


Iteration 7
Meta train Accuracy: 0.19500000029802322


Iteration 8
Meta train Accuracy: 0.21499999798834324


Iteration 9
Meta train Accuracy: 0.19500000216066837


Iteration 10
Meta train Accuracy: 0.17999999690800905


Iteration 11
Meta train Accuracy: 0.19000000040978193


Iteration 12
Meta train Accuracy: 0.2050000000745058


Iteration 13
Meta train Accuracy: 0.1900000013411045


Iteration 14
Meta train Accuracy: 0.18999999947845936


Iteration 15
Meta train Accuracy: 0.1799999987706542


Iteration 16
Meta train Accuracy: 0.19500000029802322


Iteration 17
Meta train Accuracy: 0.21000000089406967


Iteratio

# Incremental Learning

## Training model with new labels

In [ ]:
data_new = pd.read_csv(labels5)
print(data_new["Label"].unique())

In [ ]:
My_train_new = MyDataset(name="Full_data5", dataframe=data_new, label_column="Label")
print(My_train_new.__getitem__(7))

In [ ]:
meta_train_new = l2l.data.MetaDataset(My_train_new)

In [ ]:
print(len(meta_train_new))

In [ ]:
task_trans_new = [
    l2l.data.transforms.NWays(dataset=meta_train_new, n=ways),          # N-way, K-shot
    l2l.data.transforms.KShots(dataset=meta_train_new, k=shots * 2),
    l2l.data.transforms.LoadData(dataset=meta_train_new),
]
task_train_new = l2l.data.TaskDataset(dataset = meta_train_new, task_transforms = task_trans_new, num_tasks=20000)

In [ ]:
count = 0
for task in task_train_new:
    if count < 2:
        X, y = task
        print(X.shape)
        print(y.shape)
        count += 1
    else:
        break
# X, y = task11.sample()

## Meta-training and Meta-testing

In [ ]:
opt = optim.Adam(maml_dnn1.parameters(), meta_lr)
loss = nn.BCEWithLogitsLoss()

In [ ]:
print("Training model...")
train_model(maml_dnn1, meta_batch_size, task_train_new, 10000, opt, loss, adaptation_steps, shots, ways)

## Train model with new labels and some old labels

In [ ]:
data_new_old = pd.read_csv(labels5_6)
print(data_new_old["Labels"].unique())

In [ ]:
X_train_new_old, X_test_new_old = train_test_split(data_new_old, test_size=0.2)
print(X_train_new_old.shape)
print(X_test_new_old.shape)

In [ ]:
My_train_new_old = MyDataset(name="Train_data5_6", dataframe=X_train_new_old, label_column="Label")
print(My_train_new_old.__getitem__(7))

In [ ]:
meta_train_new_old = l2l.data.MetaDataset(My_train_new_old)

In [ ]:
task_trans_new_old = [
    l2l.data.transforms.NWays(dataset=meta_train_new_old, n=ways),          # N-way, K-shot
    l2l.data.transforms.KShots(dataset=meta_train_new_old, k=shots * 2),
    l2l.data.transforms.LoadData(dataset=meta_train_new_old),
]
task_train_new_old = l2l.data.TaskDataset(dataset = meta_train_new_old, task_transforms = task_trans_new_old, num_tasks=20000)

In [ ]:
My_test_new_old = MyDataset(name="Test_data5_6", dataframe=X_test_new_old, label_column="Label")
print(My_test_new_old.__getitem__(7))

In [ ]:
meta_test_new_old = l2l.data.MetaDataset(My_test_new_old)

In [ ]:
task_test_new_old = [
    l2l.data.transforms.NWays(dataset=meta_test_new_old, n=ways),
    l2l.data.transforms.KShots(dataset=meta_test_new_old, k=shots * 2),
    l2l.data.transforms.LoadData(dataset=meta_test_new_old),
]
task_test_new_old = l2l.data.TaskDataset(dataset = meta_test_new_old, task_transforms = task_test_new_old, num_tasks=100)


In [ ]:
count = 0
for task in task_train_new_old:
    if count < 2:
        X, y = task
        print(X.shape)
        print(y.shape)
        count += 1
    else:
        break
# X, y = task11.sample()

In [ ]:
count = 0
for task in task_test_new_old:
    if count < 2:
        X, y = task
        print(X.shape)
        print(y.shape)
        count += 1
    else:
        break
# X, y = task11.sample()

In [ ]:
opt = optim.Adam(maml_dnn1.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

In [ ]:
print("Training model...")
train_model(maml_dnn1, meta_batch_size, task_train_new_old, 10000, opt, loss, adaptation_steps, shots, ways)

In [ ]:
print("Testing model...")
test_model(maml_dnn1, meta_batch_size, task_test_new_old, loss, adaptation_steps, shots, ways)